In [14]:
from common import *
import pandas as pd
from scipy.sparse import save_npz, load_npz
from src.representations import DoubleTfIdfVectorizer, SpacyNEClassifier
import os


In [1]:
from src.representations import DoubleTfIdfVectorizer, SpacyNEClassifier,BioTfIdfVectorizer
import pandas as pd
t3 = "He has worked in Mexico and Uruguay."
t4 = "He is the UK Independence Party's weapon"
t2 = "Rick Morris is a winner."
t1 = "UKIP's secret weapon?"
texts = [t2, t4]
docs = [
    ["Rick", "Morris", "is", "a",  "winner","."],
    ["He","is","the","UK","Independence","Party","'s","weapon","."]
]
tags = [
    ["B-PERSON", "I-PERSON", "O", "O",  "O","O"],
    ["O","O","O","B-GPE","B-NORP","I-NORP","I-NORP","O","O"]
]
ner = SpacyNEClassifier()
vect = BioTfIdfVectorizer(ner,
            filter_stopwords = False,
            filter_punctuation=False,
            lemmatize = False,
            normalize = False,
            fix_contractions = False)

df = pd.DataFrame(vect.fit_transform(tokenized=pd.Series(docs), bio_tags= pd.Series(tags)).toarray(), columns = vect.get_feature_names())
df.shape

[IN] Tokens:['Rick', 'Morris', 'is', 'a', 'winner', '.'] Tags: ['B-PERSON', 'I-PERSON', 'O', 'O', 'O', 'O']
[OUT] Tokens:['rick', 'morris', 'is', 'a', 'winner', '.'] Tags: ['B-PERSON', 'I-PERSON', 'O', 'O', 'O', 'O']
[IN] Tokens:['He', 'is', 'the', 'UK', 'Independence', 'Party', "'s", 'weapon', '.'] Tags: ['O', 'O', 'O', 'B-GPE', 'B-NORP', 'I-NORP', 'I-NORP', 'O', 'O']
[OUT] Tokens:['he', 'is', 'the', 'uk', 'independence', 'party', "'s", 'weapon', '.'] Tags: ['O', 'O', 'O', 'B-GPE', 'B-NORP', 'I-NORP', 'I-NORP', 'O', 'O']
[IN] Tokens:['Rick', 'Morris', 'is', 'a', 'winner', '.'] Tags: ['B-PERSON', 'I-PERSON', 'O', 'O', 'O', 'O']
[OUT] Tokens:['rick', 'morris', 'is', 'a', 'winner', '.'] Tags: ['B-PERSON', 'I-PERSON', 'O', 'O', 'O', 'O']
Terms: ['rick', 'PERSON', 'morris', 'PERSON', 'is', 'a', 'winner', '.']
Feature counts: {'rick': 1, 'PERSON': 2, 'morris': 1, 'is': 2, 'a': 1, 'winner': 1, '.': 2, 'he': 1, 'the': 1, 'uk': 1, 'GPE': 1, 'independence': 1, 'party': 1, "'s": 1, 'weapon': 1}


Count vocab, row ['Rick', 'Morris', 'is', 'a', 'winner', '.'], tokens: ['rick', 'morris', 'is', 'a', 'winner', '.']
Count vocab, row ['B-PERSON', 'I-PERSON', 'O', 'O', 'O', 'O'], tags: ['B-PERSON', 'I-PERSON', 'O', 'O', 'O', 'O']
Count vocab, keys: ['rick', 'PERSON', 'morris', 'PERSON', 'is', 'a', 'winner', '.']
Count vocab, row ['He', 'is', 'the', 'UK', 'Independence', 'Party', "'s", 'weapon', '.'], tokens: ['he', 'is', 'the', 'uk', 'independence', 'party', "'s", 'weapon', '.']
Count vocab, row ['O', 'O', 'O', 'B-GPE', 'B-NORP', 'I-NORP', 'I-NORP', 'O', 'O'], tags: ['O', 'O', 'O', 'B-GPE', 'B-NORP', 'I-NORP', 'I-NORP', 'O', 'O']
Count vocab, keys: ['he', 'is', 'the', 'uk', 'GPE', 'independence', 'NORP', 'party', 'NORP', "'s", 'NORP', 'weapon', '.']


(2, 15)

In [2]:
c  = vect.get_feature_names()
c.sort()
c

["'s",
 '.',
 'GPE',
 'PERSON',
 'a',
 'he',
 'independence',
 'is',
 'morris',
 'party',
 'rick',
 'the',
 'uk',
 'weapon',
 'winner']

In [2]:
def df_to_latex (df: pd.DataFrame, label:str, caption):
    latex = "\\begin{table}[!h] \label{" +label+ "} \centering\n" \
            "\caption{" + caption + "}" \
            "\\begin{tabular}{"
    headers = ['Dokument 1','Dokument 2']
    latex += '| c ' * (len(headers) + 1)
    latex += '|}\hline \n'
    latex += "Tokeny & " + " & ".join(headers)  + "\\\\ \hline \n"

    for col in df.columns:
        latex += col + "&"
        for h_idx in range(len(headers)):
            latex +=  "{0:.3f}".format(df.iloc[h_idx].loc[col]) + " & "
        latex = latex[:-2]  #delete last &
        latex += "\\\\ \hline\n"

    latex += "\end{tabular}\n\end{table}\n"
    return latex

print(df_to_latex(df, "tab:tfidf-bio","Reprezentacja wektorowa bio"))

\begin{table}[!h] \label{tab:tfidf-bio} \centering
\caption{Reprezentacja wektorowa bio}\begin{tabular}{| c | c | c |}\hline 
Tokeny & Dokument 1 & Dokument 2\\ \hline 
PERSON&0.333 & 0.000 \\ \hline
rick&0.119 & 0.000 \\ \hline
morris&0.119 & 0.000 \\ \hline
is&0.167 & 0.111 \\ \hline
a&0.119 & 0.000 \\ \hline
winner&0.119 & 0.000 \\ \hline
.&0.167 & 0.111 \\ \hline
he&0.000 & 0.079 \\ \hline
the&0.000 & 0.079 \\ \hline
uk&0.000 & 0.079 \\ \hline
GPE&0.000 & 0.079 \\ \hline
independence&0.000 & 0.079 \\ \hline
party&0.000 & 0.079 \\ \hline
's&0.000 & 0.079 \\ \hline
weapon&0.000 & 0.079 \\ \hline
\end{tabular}
\end{table}



In [22]:
def df_to_latex_old (df: pd.DataFrame, label:str, caption):
    latex = "\\begin{table}[!h] \label{" +label+ "} \centering\n" \
            "\caption{" + caption + "}" \
            "\\begin{tabular}{"
    cols = df.columns.tolist()
    latex += '| c ' * (len(cols) + 1)
    latex += '|}\hline \n'
    latex += "Zbiór danych & " + " & ".join(cols)  + "\\\\ \hline \n"

    for index,row in df.iterrows():
        latex += str(index) + " & "
        for c in cols:
            latex +=  "{0:.2f}".format(row[c]) + " & "
        latex = latex[:-2]  #delete last &
        latex += "\\\\ \hline\n"

    latex += "\end{tabular}\n\end{table}\n"
    return latex
print(df_to_latex_old(df, "tab:tfidf-bio","Reprezentacja wektorowa bio"))

\begin{table}[!h] \label{tab:tfidf-bio} \centering
\caption{Reprezentacja wektorowa bio}\begin{tabular}{| c | c | c | c | c | c | c | c | c | c | c | c | c |}\hline 
Zbiór danych & PERSON & rick & morris & is & a & winner & he & the & uk & independence & party & weapon\\ \hline 
0 & 0.33 & 0.12 & 0.12 & 0.17 & 0.12 & 0.12 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 \\ \hline
1 & 0.00 & 0.00 & 0.00 & 0.12 & 0.00 & 0.00 & 0.09 & 0.09 & 0.09 & 0.09 & 0.09 & 0.09 \\ \hline
\end{tabular}
\end{table}



In [9]:
def load_raw(data: Dataset, name):
    return pd.read_csv(DATA_DIR + data.value + "\\raw\\" + name + ".csv", nrows = 20)

In [10]:
ner = SpacyNEClassifier()
double_vect = DoubleTfIdfVectorizer(ner_clf=ner, min_df=1,
        max_df=20)
df = load_raw(Dataset.BBC, "train")

res = double_vect.fit_transform(df['text'])

In [11]:
save_npz(DATA_DIR + "npz-try.npz", res)


In [12]:
columns_df = pd.DataFrame({'col': double_vect.get_feature_names()})
columns_df.to_csv(DATA_DIR+ "npz-column.csv", index = False)

In [17]:
def save_as_npz (dataset: Dataset, state: State, name, vectorizer, df_raw: pd.DataFrame, sp_array):
    save_npz(DATA_DIR + dataset.value + "\\" + state.value + "\\" + "array-" + name + ".npz", sp_array)
    df_raw["target"].to_csv(DATA_DIR + dataset.value + "\\" + state.value + "\\" + "target-" + name + ".csv", index=False)
    validate_or_save_columns(dataset, state, vectorizer)

def validate_or_save_columns (dataset: Dataset, state: State, vectorizer):
    name = DATA_DIR + dataset.value + "\\" + state.value + "\\columns.csv"
    col_df = pd.DataFrame({"columns": vectorizer.get_feature_names()})
    if os.path.exists(name):
        cols = pd.read_csv(name)
        assert cols.shape == col_df.shape
    else:
        col_df.to_csv(name, index = False)

def read_as_dataframe(dataset: Dataset, state: State, name):
    data_dir = DATA_DIR + dataset.value + "\\" + state.value + "\\"
    sp_array = load_npz(data_dir + "array-" + name + ".npz" )
    cols = pd.read_csv(data_dir + "columns.csv")
    target = pd.read_csv(data_dir +  "target-" + name + ".csv")

    # sanity check
    assert len(cols) == sp_array.shape[1]
    assert len(target) == sp_array.shape[0]
    df = pd.DataFrame(sp_array.toarray(), columns=cols['columns'])
    df['TARGET'] = target['target']
    return df


In [18]:
save_as_npz(Dataset.AG_NEWS, State.BIO, "test-save-npz",double_vect, df, res)
read_as_dataframe(Dataset.AG_NEWS, State.BIO, "test-save-npz")

In [20]:
df2 = read_as_dataframe(Dataset.AG_NEWS, State.BIO, "test-save-npz")

print(df2.head())

columns,offer,video,demand,service,programme,tv,sky_NE,telewest_NE,pvr_NE,cable,...,dispute,radio_NE,spectator,butt,bigley_NE,edit,yes,angry,semitic_NE,TARGET
0,-0.069626,-0.078256,-0.071983,-0.052171,-0.057441,-0.069316,-0.067801,-0.067801,-0.067801,-0.067501,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,tech
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,politics
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,sport
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,tech
4,-0.006573,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,entertainment


In [22]:
df2.columns

Index(['offer', 'video', 'demand', 'service', 'programme', 'tv', 'sky_NE',
       'telewest_NE', 'pvr_NE', 'cable',
       ...
       'dispute', 'radio_NE', 'spectator', 'butt', 'bigley_NE', 'edit', 'yes',
       'angry', 'semitic_NE', 'TARGET'],
      dtype='object', name='columns', length=2027)

False

38